VADER-Based Sentiment Analysis on Tweets V/S Roberta based transformer from hugging face for sentiment analysis using NLP



1.first we are going to start with VADER-Based sentiment analysis

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

plt.style.use("ggplot")

In [ ]:
# Load the dataset
df = pd.read_csv("twitter_data_500.csv", header=None, names=["textID", "text"])
print(df.iloc[3:])

In [ ]:
nltk.download("vader_lexicon")

In [ ]:
# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Analyze each tweet and store results
results = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['text']
    myid = row['textID']
    if isinstance(text, str):
        results[myid] = sia.polarity_scores(text)
    else:
        results[myid] = {"neg": 0, "neu": 0, "pos": 0, "compound": 0}

In [ ]:
# Convert the results to DataFrame
vader = pd.DataFrame(results).T
vader = vader.reset_index().rename(columns={"index": "textID"})
vader = vader.merge(df, on="textID", how="left")

# Labeling sentiment based on compound score
vader['Label'] = vader['compound'].apply(lambda c: 'positive' if c >= 0.05 else ('negative' if c <= -0.05 else 'neutral'))
output=vader.drop('textID', axis=1)
print(output)

In [ ]:
# Visualize sentiment distribution
fig, axs = plt.subplots(1, 3, figsize=(10, 4))
sns.barplot(data=vader, x="Label", y="pos", ax=axs[0])
sns.barplot(data=vader, x="Label", y="neu", ax=axs[1])
sns.barplot(data=vader, x="Label", y="neg", ax=axs[2])

axs[0].set_title("Positive")
axs[1].set_title("Neutral")
axs[2].set_title("Negative")
plt.tight_layout()
plt.show()


2.Now we gonna use Robverta Based transformer from Huggingface

In [ ]:
#roberta based tweet sentiment analysis
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
df1 = pd.read_csv("twitter_data_500.csv", header=None)
df1.columns = ['textID', 'text']
def polarity_scores_roberta(texts):
    neg = []
    neu = []
    pos = []
    for text in texts:
        if not isinstance(text, str) or text.strip() == "":
            neg.append(0)
            neu.append(0)
            pos.append(0)
            continue
        encoded = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            output = model(**encoded)
        scores = output.logits[0].numpy()
        scores = softmax(scores)
        neg.append(scores[0])
        neu.append(scores[1])
        pos.append(scores[2])
    return neg, neu, pos
neg, neu, pos = polarity_scores_roberta(df1["text"].tolist())
df1['roberta_neg'] = neg
df1['roberta_neu'] = neu
df1['roberta_pos'] = pos
# Display results, excluding the 'text' column
display(df1.drop('text', axis=1).head(10))